In [19]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama faiss-cpu replicate
!pip3 install pandas streamlit python-dotenv pypdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import replicate

import re
import pandas as pd
from dotenv import load_dotenv

In [21]:
load_dotenv()

True

In [22]:
df_qa = pd.read_csv("qa_base/qa.csv")

df_qa

,id,perguntas,respostas
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...


# Alucinações 

In [23]:
has_hallucination_llama = []
has_hallucination_gpt = []

## Alucinações de input

In [24]:
def has_input_hallucination(question, answer, model_name="", model=""):
    input_hallucination_prompt = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    
    if model_name == "llama":
        prompt = input_hallucination_prompt.format(question=question, answer=answer)
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(input_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"question": question, "answer": answer})
        
    return result

## Alucinações contextuais

In [35]:
def has_context_hallucination(answer, model_name="", model=""):
    context_hallucination_prompt = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Texto: Foram observadas "primeiras propagações" do vírus na população de países situados fora do continente americano, 
            assinalou o director-geral adjunto da OMS, citando o Reino Unido, o Japão e a Austrália, além do Chile, na América do Sul.

        Explicação: Chile não está fora do continente americano.
        
        Exemplo 2: 
            Texto: Ele só compra leite de vaca, pois é intolerante à lactose.

        Explicação: Se ele é intolerante a lactore, não pode beber leite de vaca.

        Exemplo 3:
            Texto: O gato miava muito alto. A temperatura hoje é de 25 graus
        
        Explicação: Os assuntos não se conectam entre si.

        Caso haja alucinação contextual, responda com 1. Caso não tenha alucinação contextual, responda com 0.
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que o texto seja "{answer}", houve alucinação contextual?
    """
    if model_name == "llama":
        prompt = context_hallucination_prompt.format(answer=answer)
        
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
        
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(context_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"answer": answer})

    return result

## Alucinações factuais

In [40]:
pdf_loader = PyPDFLoader("pdf/codigo_penal.pdf")
load_codigo_penal = pdf_loader.load()

In [41]:
codigo_penal = ""

for page in load_codigo_penal:
    codigo_penal += page.page_content

In [56]:
def vectorstore():
    txt_splitter = RecursiveCharacterTextSplitter(chunk_size=1400, 
                                                    chunk_overlap=120, 
                                                    length_function=len,
                                                    separators=["\n\n", "\n", " "])

    chunks = txt_splitter.split_documents(load_codigo_penal)
    
    embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

    vectorstore = FAISS.from_documents(chunks, 
                                       embedding=embedding)
    
    return vectorstore

In [57]:
vectorstore = vectorstore()

In [38]:
def has_factual_hallucination(answer, vectorstore, model_name="", model=""):
    factual_hallucination_prompt = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto:
        {context}

        Considerando o texto "{answer}", houve alucinação factual?
"""
    query = """Texto: {answer}"""
    retrieve = vectorstore.similarity_search(query)
     
    for retrieved in retrieve:
        context = "\n".join(str(retrieved))
    
    if model_name == "llama":
        prompt = factual_hallucination_prompt.format(context=context, answer=answer)
        
        output = replicate.run(
            "meta/llama-4-maverick-instruct",
            input={"prompt": prompt},
        )
        
        result = "".join(output).strip()
        
    elif model_name == "gpt":
        prompt = ChatPromptTemplate.from_template(factual_hallucination_prompt)
        chain = prompt | model
        result = chain.invoke({"context":context, "answer": answer})

    return result

## Detecção de alucinações

In [58]:
def check_hallucinations(model_answer_df, vectorstore="", hallucination_type="", model_name = "", model=""):
    for idx, row in df_qa.iterrows():
        id = row["id"]
        question = row["perguntas"]
        answer = row["respostas"]
        
        if hallucination_type == "input":
           hallucination_check = has_input_hallucination(question, answer, model_name, model)
        elif hallucination_type == "contextual":
            hallucination_check = has_context_hallucination(answer, model_name, model)
        elif hallucination_type == "factual":
            hallucination_check = has_factual_hallucination(answer, vectorstore, model_name, model)
        
        existing_entry = next((item for item in model_answer_df if item["id"] == id), None)

        if existing_entry:
            existing_entry[f"{hallucination_type}_hallucination_llm"] = hallucination_check
        else:
            model_answer_df.append({"id":id, "question":question, "answer":answer, 
                                    f"{hallucination_type}_hallucination_reason":hallucination_check})
        
        print(hallucination_check)
        
    return hallucination_check

# Llama

### Alucinações de input

In [28]:
check_hallucinations(has_hallucination_llama, hallucination_type="input", model_name="llama")

0. A resposta está diretamente relacionada à pergunta feita, fornecendo uma explicação detalhada sobre o crime de peculato de acordo com o Código Penal, sem fugir do tópico. Portanto, não houve alucinação de input.
0, pois a resposta está diretamente relacionada à pergunta feita. A pergunta menciona "injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência" e a resposta fornece a pena prevista para esse crime, citando o artigo correspondente do Código Penal, que inclui esses elementos e outros similares. Embora a resposta mencione elementos adicionais não citados na pergunta (raça, cor, etnia, procedência nacional), ela não foge do tópico da injúria qualificada, que é o foco da pergunta. Portanto, não há alucinação de input.
0 - A resposta está diretamente relacionada à pergunta feita, pois define e explica o conceito de feminicídio, que é o que foi questionado. Portanto, não houve alucinação de input, pois a resposta permanece no t

'1, pois a resposta fornecida não está diretamente relacionada à pergunta feita sobre o crime doloso, mas sim aborda o conceito de crime culposo e menciona uma situação não relacionada. A pergunta busca uma caracterização do crime doloso, enquanto a resposta discorre sobre crime culposo e uma situação específica não relacionada ao tema. Isso caracteriza uma alucinação de input, pois a resposta foge do tópico da pergunta.'

In [29]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

### Alucinações contextuais

In [32]:
check_hallucinations(has_hallucination_llama, hallucination_type="contextual", model_name="llama")

0, pois o texto apresenta uma explicação coerente sobre o crime de peculato, citando o artigo correspondente do Código Penal, sem contradições ou assuntos desconexos. A descrição do crime e as condições em que ele ocorre estão alinhadas com a definição legal apresentada.
0. Não há alucinação contextual no texto fornecido, pois a informação apresentada está de acordo com o Art. 140, § 3º, do Código Penal brasileiro, que trata da pena para injúria qualificada, incluindo a utilização de elementos referentes à religião, raça, cor, etnia, procedência nacional ou condição de pessoa idosa ou com deficiência. A pena mencionada está coerente com o dispositivo legal citado.
0. Não houve alucinação contextual.

A explicação para isso é que o texto apresenta uma definição de feminicídio e sua caracterização de acordo com o Código Penal, sem contradições ou informações desconexas. O texto discorre sobre o tema de forma coerente, apresentando as condições sob as quais um crime de homicídio é conside

'0. Não houve alucinação contextual.\n\nA explicação é que o texto apresenta uma definição de crime culposo e, em seguida, faz uma afirmação sobre um agente policial investigando fatos. Embora os assuntos estejam relacionados ao contexto jurídico e policial, não há uma contradição direta entre as duas afirmações. O crime culposo é um conceito jurídico e a investigação do agente policial é uma ação que pode estar relacionada a um crime, incluindo o culposo. Portanto, não há alucinação contextual evidente nesse caso.'

In [34]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

df_hallucination_llama

,id,question,answer,input_hallucination_reason,contextual_hallucination_llm
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...,0. A resposta está diretamente relacionada à p...,"0, pois o texto apresenta uma explicação coere..."
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...,"0, pois a resposta está diretamente relacionad...",0. Não há alucinação contextual no texto forne...
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...,0 - A resposta está diretamente relacionada à ...,0. Não houve alucinação contextual.\n\nA expli...
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...,"0, pois a resposta está diretamente relacionad...","0, não houve alucinação contextual, pois o tex..."
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...,0 A resposta está diretamente relacionada à pe...,0. O texto apresenta uma definição jurídica co...
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...,"1, pois a resposta foge do tópico da pergunta ...","0. Não houve alucinação contextual, pois o tex..."
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...,"1, pois a resposta fornecida não está relacion...","0, pois a frase apresenta uma informação coere..."
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...,0 A resposta não apresenta alucinação de input...,"0, pois o texto apresenta uma informação espec..."
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...,"1, pois a resposta não está relacionada ao cri...","0, pois a frase apresenta uma informação coere..."
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...,"0, pois a resposta aborda o tema da pergunta, ...",0. Não houve alucinação contextual.\n\nA expli...


### Alucinações factuais

In [59]:
check_hallucinations(has_hallucination_llama, vectorstore=vectorstore, hallucination_type="factual", model_name="llama")

0 Não houve alucinação factual. O texto apresentado parece ser uma descrição do crime de peculato de acordo com o Código Penal brasileiro, e o artigo mencionado (Art. 312) corresponde ao artigo do Código Penal que trata desse crime. A descrição está de acordo com a definição legal do crime de peculato. Portanto, não há informações factuais falsas no texto apresentado.
0. Não houve alucinação factual.

O texto fornecido como contexto apresenta uma citação de um artigo do código penal ("Art. 246. Deixar, sem justa causa, de prover à instrução primária de filho em idade escolar: Pena – detenção, de quinze dias a um mês, ou multa.") que não está diretamente relacionado à afirmação sobre a pena prevista para injúria quando há utilização de elementos referentes à religião, raça, cor, etnia, procedência nacional ou condição de pessoa idosa ou com deficiência.

No entanto, a afirmação em si ("A pena prevista para injúria quando há utilização de elementos referentes à religião, raça, cor, etnia

'1 Houve alucinação factual porque o texto fornecido não corresponde ao contexto dado. O contexto parece ser relacionado a um trecho de um código penal ou uma discussão jurídica, enquanto o texto dado discute o conceito de crime culposo e não está relacionado ao conteúdo do contexto (page_content). Além disso, o texto não é mencionado no contexto. Portanto, há uma discrepância entre o que foi fornecido e o que está no contexto.'

In [61]:
df_hallucination_llama = pd.DataFrame(has_hallucination_llama)

df_hallucination_llama

,id,question,answer,input_hallucination_reason,contextual_hallucination_llm,factual_hallucination_llm
0,1,O que caracteriza o crime de peculato?,O crime de peculato é caracterizado por apropr...,0. A resposta está diretamente relacionada à p...,"0, pois o texto apresenta uma explicação coere...",0 Não houve alucinação factual. O texto aprese...
1,2,Qual a pena prevista para injúria quando há ut...,A pena prevista para injúria quando há utiliza...,"0, pois a resposta está diretamente relacionad...",0. Não há alucinação contextual no texto forne...,0. Não houve alucinação factual.\n\nO texto fo...
2,3,O que caracteriza o feminicídio?,O feminicídio é uma qualificadora do crime de ...,0 - A resposta está diretamente relacionada à ...,0. Não houve alucinação contextual.\n\nA expli...,0 não há alucinação factual. \nO texto apresen...
3,4,Quais ações constituem o crime de extorsão?,Constituem o crime de extorsão as ações de con...,"0, pois a resposta está diretamente relacionad...","0, não houve alucinação contextual, pois o tex...","0 não houve alucinação factual, pois o texto a..."
4,5,O que caracteriza a legítima defesa?,A legítima defesa é caracterizada por repelir ...,0 A resposta está diretamente relacionada à pe...,0. O texto apresenta uma definição jurídica co...,0 Não houve alucinação factual. O texto aprese...
5,6,O que caracteriza o crime de corrupção passiva?,O crime de corrupção ativa é caracterizado por...,"1, pois a resposta foge do tópico da pergunta ...","0. Não houve alucinação contextual, pois o tex...","1 Houve alucinação factual, pois o texto forne..."
6,7,Qual é a pena para o crime de homicídio simples?,A pena para o crime de homicídio culposo é de ...,"1, pois a resposta fornecida não está relacion...","0, pois a frase apresenta uma informação coere...","1 Houve alucinação factual, pois o texto forne..."
7,8,Qual é a pena para o crime de lesão corporal c...,A pena para o crime de lesão corporal de natur...,0 A resposta não apresenta alucinação de input...,"0, pois o texto apresenta uma informação espec...","1 Houve alucinação factual, pois o texto forne..."
8,9,Qual é a pena para o crime de furto simples?,A pena para o crime de furto qualificado é de ...,"1, pois a resposta não está relacionada ao cri...","0, pois a frase apresenta uma informação coere...","1, pois o texto fornecido não menciona ""furto ..."
9,10,Como a legislação penal brasileira trata o rei...,A legislação penal estrangeira não tipifica di...,"0, pois a resposta aborda o tema da pergunta, ...",0. Não houve alucinação contextual.\n\nA expli...,0 Não houve alucinação factual. O texto aprese...


#### Veredito

In [ ]:
#df_hallucination_llama.to_csv("output/llama/has_hallucination_lama.csv")

# GPT

### Alucinações de input

### Alucinações contextuais

### Alucinações factuais

#### Veredito